# Wildfires are becoming larger and more frequent in parts of the USA

![Two maps showing an increase in larger fires in the Western US over time](https://static01.nyt.com/newsgraphics/2018/11/26/national-climate-assessment-fires/8dae3566c21240d0b274e4e0a153067a160034db/maps-Artboard-xl.jpg)

> Image source: [The International Journal of Wildland Fire via the New York Times][def]

In this analysis, I used:
  * historical fire data from the US Forest Service
  * state boundaries from the US Census office 
to generate two interactive plots:

  * Fire occurrence and maximum size by state
  * A chloropleth plot showing the total number of fires per million acres in each state

[def]: https://static01.nyt.com/newsgraphics/2018/11/26/national-climate-assessment-fires/8dae3566c21240d0b274e4e0a153067a160034db/maps-Artboard-xl.jpg

## STEP 1: BACKGROUND

In recent decades, the United States has witnessed a marked escalation in the frequency and scale of wildfires, signaling a critical shift in the nation's environmental landscape (Pierre-louis & Popovich, 2018). This concerning trend, underscores the profound influence of climate change on wildfire behavior. The continued release of greenhouse gases from various sources is poised to further amplify this peril, extending beyond the traditional wildfire-prone regions to encompass new areas, including the Southeast (Schoennagel et al., 2017). While climate change serves as a significant catalyst for these blazes, it's crucial to recognize the intertwined roles of historical land management policies and increasing human intrusion into wildland areas (Schoennagel et al., 2017). This interplay of factors demands a reevaluation of approaches to wildfire management, calling for a paradigm shift towards strategies that not only resist but also adapt to the inevitable challenges posed by a changing climate and fire regime.


Sources: 

Pierre-louis, K., &amp; Popovich, N. (2018, November 27). Climate change is fueling wildfires nationwide, New Report warns. The New York Times. https://www.nytimes.com/interactive/2018/11/27/climate/wildfire-global-warming.html 

Schoennagel, T., Balch, J. K., Brenkert-Smith, H., Dennison, P. E., Harvey, B. J., Krawchuk, M. A., Mietkiewicz, N., Morgan, P., Moritz, M. A., Rasker, R., Turner, M. G., &amp; Whitlock, C. (2017). Adapt to more wildfire in western North American forests as climate changes. Proceedings of the National Academy of Sciences, 114(18), 4582–4590. https://doi.org/10.1073/pnas.1617464114 



  * [The New York Times on increases in large wildfires in the Western US](https://www.nytimes.com/interactive/2018/11/27/climate/wildfire-global-warming.html)
  * [Adapt to more wildfire in western North American forests as climate changes](https://www.pnas.org/doi/abs/10.1073/pnas.1617464114), contributed to by former Earthlab director Jennifer Balch and many others

In [1]:
import os
import warnings

import earthpy as et
import geopandas as gpd
import geoviews as gv
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import pandas as pd
import pyogrio

warnings.filterwarnings('ignore')

## STEP 2: SET UP

In the cell below, add code to import the packages/modules needed to:

* download data from URLs - we recommend using the `earthpy` package for this notebook as it handles a lot of this process
* create reproducible file paths
* work with DataFrames
* work with GeoDataFrames
* (optional) suppress warnings - if you wish to do this, include the code to do so at the bottom of this cell
* (optional) create interactive plots - note that you will probably need holoviews and geoviews in addition to the pandas extension to customize plots the way we suggest

Be sure to list the package imports following the appropriate PEP 8 order. 

In [2]:
# Download the states data
states_url = "https://www2.census.gov/geo/tiger/TIGER2020/STATE/tl_2020_us_state.zip"
states_dir = et.data.get_data(url=states_url)
states_path = os.path.join(states_dir, 'tl_2020_us_state.shp')
states_gdf = gpd.read_file(states_path)

## Data Description and Citation 

1. Census State Data:
The TIGER/Line dataset is a comprehensive geospatial dataset provided by the U.S. Census Bureau. It includes information about various geographic entities within the United States, such as states, counties, cities, and more. The dataset is widely used for demographic, economic, and geographic analysis and forms the basis for various Census Bureau products. This specific dataset, TIGER/Line Sd, likely includes additional demographic and economic data associated with the geographic features, providing valuable information for a wide range of applications.

2. Data Source Citation:

Dataset Name: TIGER/Line® with Selected Demographic and Economic Data (TIGER/Line Sd).
Data Source: U.S. Census Bureau, Geography Division.
Year: 2020.
URL: https://www2.census.gov/geo/tiger/TIGER2020/STATE/tl_2020_us_state.zip

In [3]:
# Download the fire data (assuming you already have the URL)
fire_url = "https://www.fs.usda.gov/rds/archive/products/RDS-2013-0009.6/RDS-2013-0009.6_Data_Format2_GDB.zip"
fire_dir = et.data.get_data(url=fire_url)
fire_path = os.path.join(fire_dir, 'Data', 'FPA_FOD_20221014.gdb')
fire_gdf = pyogrio.read_dataframe(fire_path, layer='Fires')

In [4]:
# Clean and process the fire data (same as before)
fire_clean_gdf = (
    fire_gdf
    [['FOD_ID' , 'DISCOVERY_DATE' , 'FIRE_SIZE', 'geometry']]
    .set_index('FOD_ID')
)

fire_clean_gdf.DISCOVERY_DATE = pd.to_datetime(fire_clean_gdf.DISCOVERY_DATE)
fire_clean_gdf = fire_clean_gdf.to_crs(states_gdf.crs)

In [5]:
# Spatial join with states
fire_state_gdf = (
    states_gdf
    [['STUSPS', 'geometry']]
    .sjoin(fire_clean_gdf, how='inner', predicate='intersects')
)

In [6]:
# Group and aggregate data by state and year (similar to what you did with watersheds)
fire_state_gdf = (
    fire_state_gdf
    .groupby(['STUSPS', fire_state_gdf.DISCOVERY_DATE.dt.year])
    .agg(
        max_fire_size=('FIRE_SIZE', 'max'),
        num_fires=('index_right','count'))
)

In [7]:
# Compute the area of states (if needed)
states_gdf['area_ha'] = states_gdf.to_crs(9822).area / 10000 / 1000000

In [8]:
# Compute total fires in each state
fire_count_df = (
    fire_state_gdf
    .reset_index()
    [['STUSPS', 'num_fires']]
    .groupby('STUSPS')
    .sum())

In [9]:
# Add the area and geometry back in
fire_density_gdf = (
    states_gdf
    .set_index('STUSPS')
    .join(fire_count_df)
    [['num_fires', 'area_ha', 'geometry']]
)

In [10]:
# Compute fire density
fire_density_gdf['fire_density_per_ha'] = fire_density_gdf.num_fires / fire_density_gdf.area_ha

In [11]:
# Set values for y-labels and titles
labels = pd.DataFrame(dict(
    column_name=['max_fire_size', 'num_fires'],
    ylabel=['Fire Size (million ha)', 'Number of Fires'],
    title=['Largest fire on record in the region',
           'Total number of fires in the region']))
print(labels)

     column_name                  ylabel                                 title
0  max_fire_size  Fire Size (million ha)  Largest fire on record in the region
1      num_fires         Number of Fires   Total number of fires in the region


## VISUALIZE AND ANALYSE

In [12]:
# Define a function for creating multi-panel plots
def fire_plot(region_name, df=fire_state_gdf, labels=labels):
    """
    Create a multi-panel plot for a region

    Parameters
    ----------
    region_name : str
      The abbreviation of the state to generate a plot for.
    df : pd.DataFrame
      The dataframe with the data to plot. Columns much match
      an item in labels.column_name to be plotted
    labels : pd.DataFrame
      Plot labels. Must have a 'column_name', 'ylabel', and 'title'
      columns with str values. Each row will be a subplot.

    Returns
    -------
    plot : hv.core.layout.Layout
      A holoviews plot layout or similar. For use with hv.DynamicMap.
    """
    subplots = []
    for i, labs in labels.iterrows():
        subplot = (
            df.xs(region_name, level='STUSPS')
            [[labs.column_name]]
            .hvplot(
                xlabel='Year', ylabel=labs.ylabel, title=labs.title,
                width=1000, color='red', size=2
            )
        )
        subplots.append(subplot)

    plot = hv.Layout(subplots).cols(1)
    return plot

In [13]:
# Create a dropdown menu to switch between states
(
    hv.DynamicMap(
        fire_plot,
        kdims=[('region', 'Region')])
    .redim.values(region=fire_state_gdf.reset_index().STUSPS)
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'139c2763-db3a-4c6b-b9f9-cf957e59bf70': {'version…

### Chloropleth plot of fire density

Create a **chloropleth** map of the number of fires per million acres in each state in the US. A **chloropleth** map shows each shape colored by a value.

EXTRA CHALLENGE: The Alaska Region crosses over the 0 longitude line, which causes the plot to display strangely. Can you figure out how to get your Cloropleth centered? (5 pts extra credit)

In [14]:
fire_density_gdf.geometry = fire_density_gdf.geometry.simplify(tolerance=.1)

In [15]:
import geodatasets as gds
#gv.Polygons(
(
    fire_density_gdf
    .reset_index()
    [['fire_density_per_ha', 'geometry']])
 
poly_plot = (
    gv.Polygons(
        fire_density_gdf
            .reset_index()
            .dropna()
            [['fire_density_per_ha','geometry']])
    .opts(
        width=1000, height=600,
        colorbar= True, color='fire_density',
        cmap='plasma', line_color='white',
        xaxis='bare', yaxis='bare',tools=['hover'],
    )
)
gv.tile_sources.OSM * poly_plot

:Overlay
   .WMTS.I     :WMTS   [Longitude,Latitude]
   .Polygons.I :Polygons   [Longitude,Latitude]   (fire_density_per_ha)